In [1]:
import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy as np  # Import Numpy number tools
from pyopencl import array as clarray
for platform in cl.get_platforms():
    print('=' * 60)
    print('Platform - Name:  ' + platform.name)
    print('Platform - Vendor:  ' + platform.vendor)
    print('Platform - Version:  ' + platform.version)
    print('Platform - Profile:  ' + platform.profile)
    # Print each device per-platform
    for device in platform.get_devices():
        print('    ' + '-' * 56)
        print('    Device - Name:  ' + device.name)
        print('    Device - Type:  ' + cl.device_type.to_string(device.type))
        print('    Device - Max Clock Speed:  {0} Mhz'.format(device.max_clock_frequency))
        print('    Device - Compute Units:  {0}'.format(device.max_compute_units))
        print('    Device - Local Memory:  {0:.0f} KB'.format(device.local_mem_size/1024.0))
        print('    Device - Constant Memory:  {0:.0f} KB'.format(device.max_constant_buffer_size/1024.0))
        print('    Device - Global Memory: {0:.0f} GB'.format(device.global_mem_size/1073741824.0))
        print('    Device - Max Buffer/Image Size: {0:.0f} MB'.format(device.max_mem_alloc_size/1048576.0))
        print('    Device - Max Work Group Size: {0:.0f}'.format(device.max_work_group_size))
print('\n')
platform = cl.get_platforms()[0]  # Select the first platform [0]
device = platform.get_devices()[1]  # Select the first device on this platform [0]
print(device)
context = cl.Context([device])
#context = cl.create_some_context()  # Initialize the Context
queue = cl.CommandQueue(context)  # Instantiate a Queue

Platform - Name:  Apple
Platform - Vendor:  Apple
Platform - Version:  OpenCL 1.2 (Oct 29 2018 21:43:16)
Platform - Profile:  FULL_PROFILE
    --------------------------------------------------------
    Device - Name:  Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
    Device - Type:  CPU
    Device - Max Clock Speed:  2900 Mhz
    Device - Compute Units:  4
    Device - Local Memory:  32 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 8 GB
    Device - Max Buffer/Image Size: 2048 MB
    Device - Max Work Group Size: 1024
    --------------------------------------------------------
    Device - Name:  Intel(R) Iris(TM) Graphics 550
    Device - Type:  GPU
    Device - Max Clock Speed:  1050 Mhz
    Device - Compute Units:  48
    Device - Local Memory:  64 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 2 GB
    Device - Max Buffer/Image Size: 384 MB
    Device - Max Work Group Size: 256


<pyopencl.Device 'Intel(R) Iris(TM) Graphics 550' on 'Apple' a

In [2]:
program = cl.Program(context, """
#ifdef cl_khr_byte_addressable_store
#pragma OPENCL EXTENSION cl_khr_byte_addressable_store : disable
#endif

#ifdef cl_nv_pragma_unroll
#define NVIDIA
#pragma OPENCL EXTENSION cl_nv_pragma_unroll : enable
#endif

#ifdef NVIDIA
inline uint SWAP32(uint x)
{
	x = rotate(x, 16U);
	return ((x & 0x00FF00FF) << 8) + ((x >> 8) & 0x00FF00FF);
}
#else
#define SWAP32(a)	(as_uint(as_uchar4(a).wzyx))
#endif

#define K0  0x5A827999
#define K1  0x6ED9EBA1
#define K2  0x8F1BBCDC
#define K3  0xCA62C1D6

#define H1 0x67452301
#define H2 0xEFCDAB89
#define H3 0x98BADCFE
#define H4 0x10325476
#define H5 0xC3D2E1F0

#ifndef uint32_t
#define uint32_t unsigned int
#endif

uint32_t SHA1CircularShift(int bits, uint32_t word)
{
	return ((word << bits) & 0xFFFFFFFF) | (word) >> (32 - (bits));
}

__kernel void sha1_crypt_kernel(__global uint *data_info,__global char *plain_key,  __global uint *digest){
    int t, gid, msg_pad;
    int stop, mmod;
    uint i, ulen, item, total;
    uint W[80], temp, A,B,C,D,E;
    uint num_keys = data_info[1];
	int current_pad;

	msg_pad=0;

	ulen = data_info[2];
	total = ulen%64>=56?2:1 + ulen/64;



    digest[0] = 0x67452301;
	digest[1] = 0xEFCDAB89;
	digest[2] = 0x98BADCFE;
	digest[3] = 0x10325476;
	digest[4] = 0xC3D2E1F0;
	for(item=0; item<total; item++)
	{

		A = digest[0];
		B = digest[1];
		C = digest[2];
		D = digest[3];
		E = digest[4];

	#pragma unroll
		for (t = 0; t < 80; t++){
		W[t] = 0x00000000;
		}
		msg_pad=item*64;
		if(ulen > msg_pad)
		{
			current_pad = (ulen-msg_pad)>64?64:(ulen-msg_pad);
		}
		else
		{
			current_pad =-1;		
		}

	
		if(current_pad>0)
		{
			i=current_pad;

			stop =  i/4;
			
			for (t = 0 ; t < stop ; t++){
				W[t] = ((uchar)  plain_key[msg_pad + t * 4]) << 24;
				W[t] |= ((uchar) plain_key[msg_pad + t * 4 + 1]) << 16;
				W[t] |= ((uchar) plain_key[msg_pad + t * 4 + 2]) << 8;
				W[t] |= (uchar)  plain_key[msg_pad + t * 4 + 3];
			
			}
			mmod = i % 4;
			if ( mmod == 3){
				W[t] = ((uchar)  plain_key[msg_pad + t * 4]) << 24;
				W[t] |= ((uchar) plain_key[msg_pad + t * 4 + 1]) << 16;
				W[t] |= ((uchar) plain_key[msg_pad + t * 4 + 2]) << 8;
				W[t] |=  ((uchar) 0x80) ;
			} else if (mmod == 2) {
				W[t] = ((uchar)  plain_key[msg_pad + t * 4]) << 24;
				W[t] |= ((uchar) plain_key[msg_pad + t * 4 + 1]) << 16;
				W[t] |=  0x8000 ;
			} else if (mmod == 1) {
				W[t] = ((uchar)  plain_key[msg_pad + t * 4]) << 24;
				W[t] |=  0x800000 ;
			} else /*if (mmod == 0)*/ {
				W[t] =  0x80000000 ;
			}
			
			if (current_pad<56)
			{
				W[15] =  ulen*8 ;
			
			}
		}
		else if(current_pad <0)
		{
			if( ulen%64==0)
				W[0]=0x80000000;
			W[15]=ulen*8;
		
		}

		

		for (t = 16; t < 80; t++)
		{
			W[t] = SHA1CircularShift(1, W[t - 3] ^ W[t - 8] ^ W[t - 14] ^ W[t - 16]);
		}

		for (t = 0; t < 20; t++)
		{
			temp = SHA1CircularShift(5, A) +
				((B & C) | ((~B) & D)) + E + W[t] + K0;
			temp &= 0xFFFFFFFF;
			E = D;
			D = C;
			C = SHA1CircularShift(30, B);
			B = A;
			A = temp;
		}

		for (t = 20; t < 40; t++)
		{
			temp = SHA1CircularShift(5, A) + (B ^ C ^ D) + E + W[t] + K1;
			temp &= 0xFFFFFFFF;
			E = D;
			D = C;
			C = SHA1CircularShift(30, B);
			B = A;
			A = temp;
		}

		for (t = 40; t < 60; t++)
		{
			temp = SHA1CircularShift(5, A) +
				((B & C) | (B & D) | (C & D)) + E + W[t] + K2;
			temp &= 0xFFFFFFFF;
			E = D;
			D = C;
			C = SHA1CircularShift(30, B);
			B = A;
			A = temp;
		}

		for (t = 60; t < 80; t++)
		{
			temp = SHA1CircularShift(5, A) + (B ^ C ^ D) + E + W[t] + K3;
			temp &= 0xFFFFFFFF;
			E = D;
			D = C;
			C = SHA1CircularShift(30, B);
			B = A;
			A = temp;
		}

		digest[0] = (digest[0] + A) & 0xFFFFFFFF;
		digest[1] = (digest[1] + B) & 0xFFFFFFFF;
		digest[2] = (digest[2] + C) & 0xFFFFFFFF;
		digest[3] = (digest[3] + D) & 0xFFFFFFFF;
		digest[4] = (digest[4] + E) & 0xFFFFFFFF;


	}
}""").build()  # Create the OpenCL program

/Users/aadebuger/GEXT/openclvenv3/lib/python3.6/site-packages/pyopencl/__init__.py:233: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [34]:
b0 = np.array([0x61,0x0], dtype=np.byte)
text='a'
b0=np.fromstring(text, dtype=np.byte)
b=pycl_array.to_device(queue,b0)

/Users/aadebuger/GEXT/openclvenv3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
a0=np.array([0x40,0x5,len(text)],dtype=np.int32)
a=pycl_array.to_device(queue,a0)

In [36]:
c = clarray.empty(queue, (5,), np.int32)

In [37]:
program.sha1_crypt_kernel(queue,(10,), None,a.data,b.data,c.data)

In [7]:
print("c: {}".format(c)) 

c: [  896145707  2031333452  1415007512 -1030928666   961816747]


In [8]:
print("a: {}".format(a))

a: [64  5  1]


In [9]:
print("b: {}".format(b))

b: [49  0]


In [10]:
print(hex(c[0]))

TypeError: 'Array' object cannot be interpreted as an integer

In [28]:
print("%x"%(c[0,]))

TypeError: %x format: an integer is required, not Array

In [20]:
print(bin(c[0]))

TypeError: 'Array' object cannot be interpreted as an integer

In [25]:
print(hex(100))

0x64


In [27]:
print(hex(896145707))

0x356a192b


In [34]:
print(hex(c[0:1].top))

AttributeError: 'Array' object has no attribute 'top'

In [30]:
c[0].type


AttributeError: 'Array' object has no attribute 'type'

In [35]:
print("c: {}".format(c)) 

c: [  896145707  2031333452  1415007512 -1030928666   961816747]


In [11]:
for item in c:
        print(item)

896145707
2031333452
1415007512
-1030928666
961816747


In [43]:
print(c.get())

[  896145707  2031333452  1415007512 -1030928666   961816747]


In [27]:
for item in c.get():
        print(hex(item&0xffffffff))

0x86f7e437
0xfaa5a7fc
0xe15d1ddc
0xb9eaeaea
0x377667b8


In [28]:
import binassci

ModuleNotFoundError: No module named 'binassci'

In [38]:
import binascii
strlist=[]
for item in c.get():
        print(  hex(item&0xffffffff))
        strlist.append(hex(item&0xffffffff)[2:])
''.join(strlist)

0x86f7e437
0xfaa5a7fc
0xe15d1ddc
0xb9eaeaea
0x377667b8


'86f7e437faa5a7fce15d1ddcb9eaeaea377667b8'